In [11]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd
import os
import dotenv

dotenv.load_dotenv(override=True)

False

In [12]:
OUTPUT_DIR = "../../gemini-benchmark/outputs/bbh"

In [13]:
models = os.listdir(OUTPUT_DIR)

In [14]:
models.remove(".ipynb_checkpoints")
# models.remove("mixtral")

In [15]:
models

['gpt-4-1106-preview', 'mixtral', 'gpt-3.5-turbo', 'gemini-pro']

In [16]:
# Upload base dataset
df = pd.read_json(os.path.join(OUTPUT_DIR, models[0], "output.jsonl"), lines=True)
df['task_qid'] = df['task'] +'_'+ df['qid'].astype(str)
base_df = pd.DataFrame({
    "qid": df["task_qid"],
    "question": df["question"],
    "answer": df['answer'] 
})

In [17]:
zeno_client = ZenoClient("zen_P4CK880bWHV2dJLbAenb0r8Gf6QNLdTXhSkkHqPDm4I")

/data/tir/projects/tir4/users/sakter/anaconda3/envs/bleed/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package zeno-client is out of date. Your version is 0.1.13, the latest is 0.1.14.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [8]:
project = zeno_client.create_project(
    name="Gemini Evaluation - BBH",
    description="Evaluation of Gemini, GPT-4, and Mixtral on BBH dataset",
    view={
        "data": {
            "type": "text"
        },
        "label": {
            "type": "text"
        },
        "output": {
            "type": "text"
        }
    },
    public=True,
    metrics=[
        ZenoMetric(name="Accuracy Strict Match", type="mean", columns=["is_correct"]),
        ZenoMetric(name="Accuracy", type="mean", columns=["is_correct_last"])
    ],
)

Successfully updated project.
Access your project at  https://hub.zenoml.com/project/2302942e-c071-49a1-ba6d-fb9e2ee52cfe/Gemini%20Evaluation%20-%20BBH


In [9]:
project.upload_dataset(base_df, id_column="qid", data_column="question", label_column="answer")

  0%|          | 0/5 [00:00<?, ?it/s]

Successfully uploaded data


In [30]:
import re
def answer_type(answer):
    pattern = '[-+]?(?:[0-9,]*\.*\d+)'
    soln = re.findall(pattern, answer) 
    if answer.startswith('(') and answer.endswith(')'):
        return 'MCQ'
    elif answer == 'yes' or answer == 'Yes' or answer == 'No' or answer == 'no':
        return 'Yes/No'
    elif answer == 'true' or answer == 'True' or answer == 'False' or answer == 'false':
        return 'True/False'
    elif answer == 'valid' or answer == 'Valid' or answer == 'invalid' or answer == 'Invalid':
        return 'Valid/Invalid'
    elif len(soln) > 0:
        return 'Digit'
    else:
        return 'Other'

for model in models:
    df = pd.read_json(os.path.join(OUTPUT_DIR, model, "output.jsonl"), lines=True)
    df['task_qid'] = df['task'] +'_'+ df['qid'].astype(str)
    output_df = pd.DataFrame({
        "qid": df["task_qid"],
        "task": df["task"],
        "output": df.apply(lambda x: f"{x['generated_text']}\n\n{x['predict']}", axis=1),
        "output_last": df.apply(lambda x: f"{x['predict_last']}", axis=1),
        "output_type": df.apply(lambda x: f"{answer_type(x['answer'])}", axis=1),
        "question_length": df.apply(lambda x: len(x['question'].split(' ')), axis=1),
        "output_length": df.apply(lambda x: len(x['generated_text'].split(' ')), axis=1),
        "is_correct": df["is_correct"].astype(bool),
        "is_correct_last": df["is_correct_last"].astype(bool)
    })
    if model == 'gpt-4-1106-preview':
        model = 'gpt-4-turbo'
    project.upload_system(output_df, name=model, id_column="qid", output_column="output")

  0%|          | 0/8 [00:00<?, ?it/s]

Successfully uploaded system


  0%|          | 0/10 [00:00<?, ?it/s]

Successfully uploaded system


  0%|          | 0/5 [00:00<?, ?it/s]

Successfully uploaded system


  0%|          | 0/3 [00:00<?, ?it/s]

Successfully uploaded system


In [32]:


for model in models:
    df = pd.read_json(os.path.join(OUTPUT_DIR, model, "output.jsonl"), lines=True)
    df["output_type"] = df.apply(lambda x: f"{answer_type(x['answer'])}", axis=1)
    sub_df = df.loc[df['output_type'] == 'MCQ']
    df.groupby('a').count()
    
    break

                                 task  qid   
187   logical_deduction_seven_objects    0  \
188   logical_deduction_seven_objects    1   
189   logical_deduction_seven_objects    2   
190   logical_deduction_seven_objects    3   
191   logical_deduction_seven_objects    4   
...                               ...  ...   
6506                 geometric_shapes  245   
6507                 geometric_shapes  246   
6508                 geometric_shapes  247   
6509                 geometric_shapes  248   
6510                 geometric_shapes  249   

                                                 prompt   
187    The following paragraphs each describe a set ...  \
188    The following paragraphs each describe a set ...   
189    The following paragraphs each describe a set ...   
190    The following paragraphs each describe a set ...   
191    The following paragraphs each describe a set ...   
...                                                 ...   
6506   This SVG path element <path

In [55]:
df = pd.read_json(os.path.join(OUTPUT_DIR, 'gemini-pro', "output.jsonl"), lines=True)
df["output_type"] = df.apply(lambda x: f"{answer_type(x['answer'])}", axis=1)
sub_df = df.loc[df['output_type'] == 'MCQ']
# sub_df.groupby('predict_last').count()
gemini = {}
options = ['A', 'B', 'C', 'D', 'E', 'F']
for op, x in zip(options, sub_df.groupby('predict_last').count()['task'][:6]):
    gemini[op] = x/(1.0*len(sub_df))
    

In [56]:
df = pd.read_json(os.path.join(OUTPUT_DIR, 'gpt-3.5-turbo', "output.jsonl"), lines=True)
df["output_type"] = df.apply(lambda x: f"{answer_type(x['answer'])}", axis=1)
sub_df = df.loc[df['output_type'] == 'MCQ']
# sub_df.groupby('predict_last').count()['task'][:6]
gpt_3 = {}
options = ['A', 'B', 'C', 'D', 'E', 'F']
for op, x in zip(options, sub_df.groupby('predict_last').count()['task'][:6]):
    gpt_3[op] = x/(1.0*len(sub_df))

In [57]:
df = pd.read_json(os.path.join(OUTPUT_DIR, 'gpt-4-1106-preview', "output.jsonl"), lines=True)
df["output_type"] = df.apply(lambda x: f"{answer_type(x['answer'])}", axis=1)
sub_df = df.loc[df['output_type'] == 'MCQ']
gpt_4 = {}
options = ['A', 'B', 'C', 'D', 'E', 'F']
for op, x in zip(options, sub_df.groupby('predict_last').count()['task'][:6]):
    gpt_4[op] = x/(1.0*len(sub_df))

In [60]:
df = pd.read_json(os.path.join(OUTPUT_DIR, 'mixtral', "output.jsonl"), lines=True)
df["output_type"] = df.apply(lambda x: f"{answer_type(x['answer'])}", axis=1)
sub_df = df.loc[df['output_type'] == 'MCQ']
# sub_df.groupby('predict_last').count()
mixtral = {}
options = ['A', 'B', 'C', 'D', 'E', 'F']
for op, x in zip(options, sub_df.groupby('predict_last').count()['task'][:6]):
    mixtral[op] = x/(1.0*len(sub_df))

In [42]:
179/4071

0.04396954065340211

In [53]:
gemini = {
'A': 879,
'B': 962,
'C': 696,
'D': 582,
'E': 318,
'F': 135}

gpt_3 = {
    'A': 1056,
'B': 942,
'C': 743,
'D': 590,
'E': 312,
'F': 132}

In [61]:
gemini, gpt_3 , gpt_4, mixtral

({'A': 0.2159174649963154,
  'B': 0.23630557602554655,
  'C': 0.1709653647752395,
  'D': 0.14296241709653648,
  'E': 0.07811348563006633,
  'F': 0.03316138540899042},
 {'A': 0.2593957258658806,
  'B': 0.2313927781871776,
  'C': 0.18251043969540653,
  'D': 0.14492753623188406,
  'E': 0.07663964627855564,
  'F': 0.032424465733235076},
 {'A': 0.24588553181036601,
  'B': 0.24957013018914273,
  'C': 0.16850896585605502,
  'D': 0.13289118152788013,
  'E': 0.08253500368459837,
  'F': 0.03930238270695161},
 {'A': 0.20265291083271925,
  'B': 0.2164087447801523,
  'C': 0.1527880127732744,
  'D': 0.1296978629329403,
  'E': 0.050110537951363304,
  'F': 0.030459346597887498})

In [62]:
df = pd.read_json(os.path.join(OUTPUT_DIR, 'mixtral', "output.jsonl"), lines=True)
df["output_type"] = df.apply(lambda x: f"{answer_type(x['answer'])}", axis=1)
sub_df = df.loc[df['output_type'] == 'MCQ']
# sub_df.groupby('predict_last').count()
ANSWERS = {}
options = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']
for op, x in zip(options, sub_df.groupby('answer').count()['task'][:14]):
    ANSWERS[op] = x/(1.0*len(sub_df))

In [63]:
ANSWERS

{'A': 0.2380250552689757,
 'B': 0.25300908867600097,
 'C': 0.18275607958732498,
 'D': 0.13780397936624908,
 'E': 0.07246376811594203,
 'F': 0.04618029968066814,
 'G': 0.025055268975681652,
 'H': 0.0012281994595922379,
 'I': 0.009334315892901008,
 'J': 0.007369196757553427,
 'K': 0.013755833947433063,
 'L': 0.0017194792434291329,
 'M': 0.0031933185949398184,
 'N': 0.0009825595676737902}